In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import math
from ctypes import cast , POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities,IAudioEndpointVolume


cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7,min_tracking_confidence=0.7)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_,CLSCTX_ALL,None)
volume = cast(interface,POINTER(IAudioEndpointVolume))
vol_range = volume.GetVolumeRange()
min_vol = vol_range[0]
max_vol = vol_range[1]


mute_state = False
gesture_cooldown = False
cooldown_counter = 0

while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            lm_list = []
            for id,lm in enumerate(hand_landmarks.landmark):
                h,w,c = f.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lm_list.append((id,cx,cy))
            mp_drawing.draw_landmarks(f,hand_landmarks,mp_hands.HAND_CONNECTIONS)

            thumb_tip = lm_list[4][1],lm_list[4][2]
            index_tip = lm_list[8][1],lm_list[8][2]
            pinky_tip = lm_list[20][1],lm_list[20][2]

            cv2.circle(f,thumb_tip,10,(255,0,0),3)
            cv2.circle(f,index_tip,10,(255,0,0),3)
            cv2.line(f,thumb_tip,index_tip,(255,0,255),3)

            len = math.hypot(index_tip[0]-thumb_tip[0],index_tip[1]-thumb_tip[0])

            vol = np.interp(len,[30,200],[min_vol,max_vol])
            volume.SetMasterVolumeLevel(vol,None)

            vol_bar = np.interp(len,[30,200],[400,150])
            vol_perc = np.interp(len,[30,200],[0,100])

            cv2.rectangle(f,(50,180),(85,400),(0,255,0),2)
            cv2.rectangle(f,(50,int(vol_bar)),(85,400),(0,255,0),cv2.FILLED)
            cv2.putText(f,f'{int(vol_perc)}%',(40,130),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

            mute_distance = math.hypot(thumb_tip[0]-pinky_tip[0],thumb_tip[1]-pinky_tip[1])
            if mute_distance<40 and not gesture_cooldown:
                mute_state = not mute_state
                volume.SetMute(mute_state,None)
                gesture_cooldown = True

            status = "MUTED" if mute_state else "NOT MUTED"
            color = (0,0,255) if mute_state else (0,255,0)
            cv2.putText(f,status,(450,50),cv2.FONT_HERSHEY_COMPLEX,1.2,color,3)

    if gesture_cooldown:
        cooldown_counter+=1
        if cooldown_counter>20:
            gesture_cooldown=False
            cooldown_counter=0


    cv2.imshow("Video Frame",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()